# A few things about NumPy

In [ ]:
print([1,2,3] + [4,5,6])
print(np.array([1,2,3]) + np.array([4,5,6]))

a = np.zeros((3,3))
b = np.zeros((3,3))

a[1] = 1
b[:,1] = 1
c = np.arange(0,9).reshape(3,3)

print(a)
print(b)
print(c)

print(np.sum(c[1]))
print(np.sum(c[:,2]))

In [ ]:
a = np.array([1,2])
b = np.array([3,4])
np.linalg.norm(b-a, 2)

In [ ]:
a = np.array([[1, 4, 3],
              [4, 5, 6],
              [3, 6, 9]])

val, vec = np.linalg.eigh(a)

print(val)
print(np.sum(val))
print(vec)

In [ ]:
#matrix multiplication 
P = np.array([[0, 0, 1],
              [1, 0, 0],
              [0, 1, 0]])
a2 = np.dot(np.dot(P,a), P.T)
print(a2)
val2, vec2 = np.linalg.eigh(a2)
print(val2)
print(vec2)

print(np.isclose(val,val2))

In [ ]:
#elementwise multiplication
a = np.array([[1, 2],
              [3, 4]])
print(np.multiply(a,a))
print(np.dot(a,a))

# Define functions

In [ ]:
#RDKit has this function (rdkit.Chem.rdMolTransforms.GetBondLength), but just as an example:)
def get_bond_length(a1_coord, a2_coord):
    return np.linalg.norm(a1_coord - a2_coord, 2)

print(get_bond_length(  np.array([0,0]),  
                        np.array([3,4]))   )

# Exercise 1

1. Define a function that takes covalent radii, atomic numbers, and 3d coordinates and returns a connectivity matrix 

2. Define a function that takes covalent radii, atomic numbers, 3d coordinates, ('continuous' or 'discretized') and returns a Coulomb matrix and its eigenvalues (continuous or discretized)

3. Calculate connectivity matrix, discretized and continuous Coulomb matrices and their eigenvalues for the two ethane conformers.

In [ ]:
#Dalton Trans., 2008, 2832-2838.
#Covalent radii are also available in RDKit. We will use them later:)
covalent_radii = {6:0.76, 1:0.31}

#This sort of data can be easily read and processed by using cclib, pybel and rdkit, 
#but I thought that is beyond our scope today.
#Today I wanted to focus more on numpy.
#We will deal with 3d geometries using rdkit and cclib soon (next week or later)
eth1_Z = [6, 1, 1, 1, 6, 1, 1, 1]
eth1_coords = np.array([ [-0.35950280,1.56267809,-0.04562792],
[-0.00284838,0.55386809,-0.04562792],
[-0.00282996,2.06707628,-0.91927943],
[-1.42950280,1.56269128,-0.04562792],
[0.15383942,2.28863437,1.21177705],
[1.22383761,2.28692730,1.21275540],
[-0.20121714,3.29800738,1.21079966],
[-0.20442955,1.78536637,2.08542725] ])

eth2_Z = 2 * [6] + 6 * [1]
eth2_coords = np.array([[-0.35950280,1.56267809,-0.04562792],
[0.15383942,2.28863437,1.21177705],
[-0.95555359,0.72314450,0.24558815],
[0.47271315,1.22470551,-0.62708399],
[-0.95234107,2.23578574,-0.62903933],
[1.22383761,2.28692730,1.21275540],
[-0.20121714,3.29800738,1.21079966],
[-0.20442955,1.78536637,2.08542725]])

# Atom/bond features from SMILES

In [ ]:
from rdkit import Chem
mol = Chem.MolFromSmiles('CC(C)O')
print(Chem.rdmolops.GetAdjacencyMatrix(mol))
print(Chem.rdmolops.GetDistanceMatrix(mol))

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
mol

In [ ]:
#https://github.com/NREL/nfp/blob/master/nfp/preprocessing/features.py
from rdkit import Chem
def atom_features_v1(atom):
    """ Return an integer hash representing the atom type
    """

    return str((
        atom.GetSymbol(),
        atom.GetDegree(),
        atom.GetTotalNumHs(),
        atom.GetImplicitValence(),
        atom.GetIsAromatic(),
    ))


def atom_features_v2(atom):
    props = ['GetChiralTag', 'GetDegree', 'GetExplicitValence',
             'GetFormalCharge', 'GetHybridization', 'GetImplicitValence',
             'GetIsAromatic', 'GetNoImplicit', 'GetNumExplicitHs',
             'GetNumImplicitHs', 'GetNumRadicalElectrons', 'GetSymbol',
             'GetTotalDegree', 'GetTotalNumHs', 'GetTotalValence']

    atom_type = [getattr(atom, prop)() for prop in props]

    return str(tuple(atom_type))

def bond_features_v1(bond, **kwargs):
    """ Return an integer hash representing the bond type.
    
    flipped : bool
        Only valid for 'v3' version, whether to swap the begin and end atom types
    """

    return str((
        bond.GetBondType(),
        bond.GetIsConjugated(),
        bond.IsInRing(),
        sorted([
            bond.GetBeginAtom().GetSymbol(),
            bond.GetEndAtom().GetSymbol()]),
    ))


mol = Chem.MolFromSmiles('CN1CCC23C4C1CC5=C2C(=C(C=C5)O)OC3C(C=C4)O')

for atom in mol.GetAtoms():
    print(atom_features_v1(atom))
    #print(atom_features_v2(atom))

for bond in mol.GetBonds():
    print(bond_features_v1(bond))

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
mol

In [ ]:
from rdkit.Chem.rdMolDescriptors \
import CalcNumRings, CalcNumAromaticRings, CalcNumAliphaticRings

print(CalcNumRings(mol), CalcNumAromaticRings(mol), CalcNumAliphaticRings(mol))

SSSR = Chem.GetSymmSSSR(mol)
for sssr in SSSR:
    print(list(sssr))

# Assignment Week 4

One-hot encoding is one of the ways (not always the best way though) to vectorize the features that have different types (integer, float, boolean, etc.). 

(1) Write a function that takes (i) mol object, (ii) maximum number of atoms in a molecule, (iii) whether to consider explicit Hs (True or False) as input variables, and returns a list of one-hot vectors.

An one-hot vector per each atom should contain those atom features:

- If explicit_Hs = False (i.e. no AddHs) -> Symbols, Degrees, Total number of hydrogens bonded to an atom, IsInRing, IsAromatic

- If explicit_Hs = True -> Symbols(including H), Degrees, IsInRing, IsAromatic

- Zero vectors for (Max_atoms - # of atoms in a molecule) atoms

Regardless of # of atoms in a molecule, the shape of the return array should be: (Max_atoms, dimension of an one-hot vector)

(2) json-load './Week_3/exercise_2.json', and collect all the SMILES strings

(3) Apply the def written in (1) (Max_atoms = it can be determined from the largest molecule in the data file, explicit_Hs = True)

(4) How many unique atom types (i.e. one-hot vectors) in the data file?